<a href="https://colab.research.google.com/github/victorviro/Deep_learning_python/blob/master/TensorFlow%E2%80%99s_autodifferentiation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This article explains how TensorFlow’s autodifferentiation (autodiff) feature works, and how it compares to other solutions.

Suppose you define a function $f(x,y) = x^2y+y+2$, and you need its partial derivatives $\frac{\partial f}{\partial x}$ and $\frac{\partial f}{\partial y}$, typically to perform Gradient Descent (or some other optimization algorithm). Your main options are manual differentiation, finite difference approximation, forward-mode autodiff, and reverse-mode autodiff. TensorFlow implements reverse-mode autodiff, but to understand it, it’s useful to look at the other options first. So let’s go through each of them, starting with manual differentiation.

## Manual Differentiation

The first approach to compute derivatives is to pick up a pencil and a piece of paper and use your calculus knowledge to derive the appropriate equation:

$$\frac{\partial f}{\partial x}=2xy$$
$$\frac{\partial f}{\partial y}=x^2+1$$

This approach can become very tedious for more complex functions, and you run the risk of making mistakes.

## Finite Difference Approximation

Recall that the derivative $h^{\prime}(x_0)$ of a function $h(x)$ at a point $x_0$ is the slope of the function at that point. More precisely, the derivative is defined as the limit of the slope of a straight line going through this point $x_0$ and another point $x$ on the function, as $x$ gets infinitely close to $x_0$:

$$h^{\prime}(x_0) = \lim_{x \to x_0} \frac{h(x)-h(x_0)}{x-x_0} = \lim_{\epsilon \to 0} \frac{h(x_0+\epsilon)-h(x_0)}{\epsilon}$$

So, if we want to calculate the partial derivatives of $f(x, y)$ with regard to $x$ and $y$ at a point $(x,y)$ , we could compute

$$\nabla f(x,y)=(\frac{\partial f(x,y)}{\partial x},\frac{\partial f(x,y)}{\partial x}) \approx (\frac{f(x+\epsilon,y)}{\epsilon}, \frac{f(x,y+\epsilon)}{\epsilon})$$

This type of numerical approximation of the derivative is called a *finite difference approximation*, and this specific equation is called *Newton’s difference quotient*. That’s exactly what the following code does:

In [ ]:
def f(x, y):
    return x**2*y + y + 2

def derivative(f, x, y, x_eps, y_eps):
    return (f(x + x_eps, y + y_eps) - f(x, y)) / (x_eps + y_eps)

# Calculate the derivatives of f at point (3,4)
df_dx = derivative(f, 3, 4, 0.00001, 0)
df_dy = derivative(f, 3, 4, 0, 0.00001)
print(df_dx,df_dy)

24.000039999805264 10.000000000331966


Unfortunately, the result is imprecise (and it gets worse for more complicated functions). The correct results are respectively 24 and 10.

Notice that to compute both partial derivatives, we have to call `f()` at least three times (we called it four times in the preceding code, but it could be optimized). If there were 1000 parameters, we would need to call `f()` at least 1001 times. When you are dealing with large neural networks, this makes finite difference approximation way too inefficient.

However, this method is so simple to implement that it is a great tool to check that the other methods are implemented correctly. For example, if it disagrees with your manually derived function, then your function probably contains a mistake.

So far, we have considered two ways to compute gradients: using manual differentiation and using finite difference approximation. Unfortunately, both were fatally flawed to train a large-scale neural network. So let’s turn to autodiff, starting with forward mode.

## Forward-Mode Autodiff

The next figure shows how forward-mode autodiff works on an even simpler function, $g(x,y)=5+xy$. The graph for that function is represented on the left. After forward-mode autodiff, we get the graph on the right, which represents the partial derivative $\frac{\partial g}{\partial x}= 0 + (0x + y) = y$ (we could similarly obtain the partial derivative with regard to $y$).

![texto alternativo](https://i.ibb.co/2YcfWhn/fordward-mode-autodiff.png)

The algorithm will go through the computation graph from the inputs to the outputs (hence the name “forward mode”). It starts by getting the partial derivatives of the leaf nodes. The constant node (5) returns the constant 0, since the derivative of a constant is always 0. The variable $x$ returns the constant 1 since $\frac{\partial x}{\partial x} = 1$, and the variable $y$ returns the constant 0 since $\frac{\partial y}{\partial x} = 0$ (if we were looking for the partial derivative with regard to $y$, it would be the reverse).

Now we have all we need to move up the graph to the multiplication node in function $g$. Calculus tells us that the derivative of the product of two functions $u$ and $v$ is $\frac{\partial (uv)}{\partial x}=\frac{\partial v}{\partial x} u + \frac{\partial u}{\partial x} v$. We can therefore construct a large part of the graph on the right, representing $0x + y$.

Finally, we can go up to the addition node in function $g$. As mentioned, the derivative of a sum of functions is the sum of these function's derivatives. So we just need to create an addition node and connect it to the parts of the graph we have already computed. We get the correct partial derivative: 
$\frac{\partial g}{\partial x} = 0 + (0x + y)$.

However, this equation can be simplified (a lot). A few pruning steps can be applied to the computation graph to get rid of all unnecessary operations, and we get a much smaller graph with just one node: $\frac{\partial g}{\partial x} =y$. In this case simplification is fairly easy, but for a more complex function forward-mode autodiff can produce a huge graph that may be tough to simplify and lead to suboptimal performance.

Note that we started with a computation graph, and forward-mode autodiff produced another computation graph. This is called *symbolic differentiation*, and it has two nice features: first, once the computation graph of the derivative has been produced, we can use it as many times as we want to compute the derivatives of the given function for any value of $x$ and $y$; second, we can run forward-mode autodiff again on the resulting graph to get second-order derivatives if we ever need to (i.e., derivatives of derivatives). We could even compute third-order derivatives, and so on.




But it is also possible to run forward-mode autodiff without constructing a graph (i.e., numerically, not symbolically), just by computing intermediate results on the fly. One way to do this is to use dual numbers, which are weird but fascinating numbers of the form $a + b\varepsilon$, where $a$ and $b$ are real numbers and $\varepsilon$ is an infinitesimal number such that $\varepsilon^2 = 0$ (but $\varepsilon \neq 0$). You can think of the dual number $42 + 24\varepsilon$ as something like $42.0000...000024$ with an infinite number of 0s (but of course this is simplified just to give you some idea of what dual numbers are). A dual number is represented in memory as a pair of floats. For example, $42 + 24\varepsilon$ is represented by the pair $(42.0, 24.0)$.

Dual numbers can be added, multiplied, and so on, as shown in the next equation:

$$\lambda(a + b\varepsilon) = \lambda a + \lambda b\varepsilon$$
$$(a + b\varepsilon)+(c + d\varepsilon)=(a+c)+ (b+d)\varepsilon$$
$$(a + b\varepsilon)(a + b\varepsilon)=ac +(ad+bc)\varepsilon + (bd)\varepsilon^2 = ac +(ad+bc)\varepsilon$$

Most importantly, it can be shown that $h(a + b\varepsilon) = h(a) + bh^{\prime}(a)\varepsilon$, so computing $h(a + \varepsilon)$ gives you both $h(a)$ and the derivative $h^{\prime}(a)$ in just one shot. The next figure shows that the partial derivative of $f(x,y) = x^2y+y+2$ with regard to $x$ at $x = 3$ and $y = 4$ (which we will write $\frac{\partial f}{\partial x}(3,4)$) can be computed using dual numbers. All we need to do is compute $f(3 + \varepsilon, 4)$; this will output a dual number whose first component is equal to $f(3, 4)$ and whose second component is equal to $\frac{\partial f}{\partial x}(3,4)$.

![texto alternativo](https://i.ibb.co/6mt4hSv/forward-mode-autodiff-dual-numbers.png)

To compute $\frac{\partial f}{\partial x}(3,4)$ we would have to go through the graph again, but this time with $x = 3$ and $y = 4 + \varepsilon$.

So forward-mode autodiff is much more accurate than finite difference approximation, but it suffers from the same major flaw, at least when there are many inputs and few outputs (as is the case when dealing with neural networks): if there were 1000 parameters, it would require 1000 passes through the graph to compute all the partial derivatives. This is where reverse-mode autodiff shines: it can compute all of them in just two passes through the graph. Let’s see how.

## Reverse-Mode Autodiff

Reverse-mode autodiff is the solution implemented by TensorFlow. It first goes through the graph in the forward direction (i.e., from the inputs to the output) to compute the value of each node. Then it does a second pass, this time in the reverse direction (i.e., from the output to the inputs), to compute all the partial derivatives. The name "reverse mode" comes from this second pass through the graph, where gradients flow in the reverse direction. Figure D-3 represents the second pass. During the first pass, all the node values were computed, starting from $x = 3$ and $y = 4$. You can see those values at the bottom right of each node (e.g., $xx = 9$). The nodes are labeled $n_1$ to $n_7$ for clarity. The output node is $n_7$: $f(3, 4) = n_7 = 42$.

![texto alternativo](https://i.ibb.co/v4LQtnp/reverse-mode-autodiff.png)

The idea is to gradually go down the graph, computing the partial derivative of $f(x, y)$ with regard to each consecutive node, until we reach the variable nodes. For this, reverse-mode autodiff relies heavily on the chain rule, shown in the next equation.

$$\frac{\partial f}{\partial x}=\frac{\partial f}{\partial n_i}\times\frac{\partial n_i}{\partial x}$$

Since $n_7$ is the output node, $f = n_7$ so $\frac{\partial f}{\partial n_7}=1$.

Let’s continue down the graph to $n_5$: how much does $f$ vary when $n_5$ varies? The answer is $\frac{\partial f}{\partial n_5}=\frac{\partial f}{\partial n_7}\times\frac{\partial n_7}{\partial n_5}$. We already know that $\frac{\partial f}{\partial n_7}=1$, so all we need is $\frac{\partial n_7}{\partial n_5}$. Since $n_7$ simply performs the sum $n_5 + n_6$, we find that $\frac{\partial n_7}{\partial n_5}$=1, so $\frac{\partial f}{\partial n_5}=1\times 1=1$.

Now we can proceed to node $n_4$: how much does $f$ vary when $n_4$ varies? The answer is $\frac{\partial f}{\partial n_4}=\frac{\partial f}{\partial n_5}\times\frac{\partial n_5}{\partial n_4}$. Since $n_5 = n_4 \times n_2$, we find that $\frac{\partial n_5}{\partial n_4}=n_2$, so $\frac{\partial f}{\partial n_4}=1 \times n_2=4$.

The process continues until we reach the bottom of the graph. At that point we will have calculated all the partial derivatives of $f(x, y)$ at the point $x = 3$ and $y = 4$. In this example, we find $\frac{\partial f}{\partial x}=24$ and $\frac{\partial y}{\partial y}=10$. Sounds about right!

Reverse-mode autodiff is a very powerful and accurate technique, especially when there are many inputs and few outputs, since it requires only one forward pass plus one reverse pass per output to compute all the partial derivatives for all outputs with regard to all the inputs. When training neural networks, we generally want to minimize the loss, so there is a single output (the loss), and hence only two passes through the graph are needed to compute the gradients. Reverse-mode autodiff can also handle functions that are not entirely differentiable, as long as you ask it to compute the partial derivatives at points that are differentiable.

In Figure D-3, the numerical results are computed on the fly, at each node. However, that’s not exactly what TensorFlow does: instead, it creates a new computation graph. In other words, it implements symbolic reverse-mode autodiff. This way, the computation graph to compute the gradients of the loss with regard to all the parameters in the neural network only needs to be generated once, and then it can be executed over and over again, whenever the optimizer needs to compute the gradients. Moreover, this makes it possible to compute higher-order derivatives if needed.